In [1]:
import ipywidgets as widgets
from ipywidgets import interact, SelectMultiple, fixed, Checkbox, IntRangeSlider, IntSlider, FloatSlider
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import plotly.express
import plotly.graph_objects as go
from tqdm.autonotebook import tqdm
from PIL import Image
from scipy import stats
from matplotlib.image import NonUniformImage
from src.DataExtractor import DataExtractor


data_extractor = DataExtractor()

/var/folders/43/_vzgr_gs2h1ft07nv0xhftd40000gn/T/ipykernel_10126/2978709482.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# cut the image
im = Image.open(r'../figures/nhl_rink.png')
width, height = im.size

new_w = width/2
im_new = im.crop((new_w, 0, width, height))

new_width, height = im_new.size

In [3]:
year = 2017
df_season_league = data_extractor.get_season_into_dataframe('./hockey/Season' + str(year) + str(year + 1) + '/season' + str(year) + str(year + 1) + '.json')

In [109]:
def __generate_combined_shots_per_hour(df_season_league: pd.DataFrame, team_ids: list):
    total_shot_count = 0
    total_league_time_played_in_minutes = 0
    
    for team_id in tqdm(team_ids, total = len(team_ids)):
        team_season_shots_x, team_season_shots_y = data_extractor.get_team_shots_from_dataframe(df_season_league, team_id)
        total_shot_count += len(team_season_shots_x)
        
        team_season_time = data_extractor.get_time_played_from_team_season_dataframe(df_season_league, team_id)
        total_league_time_played_in_minutes += np.sum(team_season_time)
    
    total_league_time_played_in_hours = total_league_time_played_in_minutes / 60
    return total_shot_count / total_league_time_played_in_hours

    
def calculate_league_shots_per_hour(df_season_league: pd.DataFrame) -> float:
    team_ids = list(range(1,56))
    return __generate_league_shots_per_hour(df_season_league, team_ids)


def calculate_team_shots_per_hour(df_season_league: pd.DataFrame, team_id: int) -> float:
    team_ids = [team_id]
    return __generate_league_shots_per_hour(df_season_league, team_ids)


def __generate_total_shots_array(df_season_league: pd.DataFrame, team_ids: list):
    total_shots_x = np.array([])
    total_shots_y = np.array([])
    
    for team_id in tqdm(team_ids, total = len(team_ids)):
        team_season_shots_x, team_season_shots_y = data_extractor.get_team_shots_from_dataframe(df_season_league, team_id)
        total_shots_x = np.append(total_shots_x, team_season_shots_x)
        total_shots_y = np.append(total_shots_y, team_season_shots_y)
        
    return total_shots_x, total_shots_y


def get_total_league_shots_array(df_season_league: pd.DataFrame):
    team_ids = list(range(1,56))
    return __generate_total_shots_array(df_season_league, team_ids)


def get_total_team_shots_array(df_season_league: pd.DataFrame, team_id: int):
    team_ids = [team_id]
    return __generate_total_shots_array(df_season_league, team_ids)


def get_time_played_for_teams(df_season_league: pd.DataFrame, team_ids: list):
    total_league_time_played_in_minutes = 0
    
    for team_id in tqdm(team_ids, total = len(team_ids)):
        team_season_time = data_extractor.get_time_played_from_team_season_dataframe(df_season_league, team_id)
        total_league_time_played_in_minutes += np.sum(team_season_time)
        
    return total_league_time_played_in_minutes


def get_team_name(df: pd.DataFrame, team_id: int) -> str:
    df.rename(columns={
        'team.id': 'teamID', 
        'team.name': 'teamName'
        }, inplace=True)
    
    df = df[df.teamID == team_id]
    name = df.teamName
    if name.empty:
        return 'nothing'
    return name.iloc[0]
    

In [113]:
def compute_team_shots_excess_by_hour_for_team(df_season_league: pd.DataFrame, team_id: int, source, width: float, lenght: float):
    ##
    # rink = [-100,100] * [-42.5, 42.5]
    # image =  [0,550] * [0,467]
    ###
    #transformation
    ratio_w = width / 100
    ratio_l = lenght / 85

    total_team_shots_x, total_team_shots_y = get_total_team_shots_array(df_season_league, team_id)
    total_team_shots_x = np.abs(total_team_shots_x * ratio_w)
    total_team_shots_y = (total_team_shots_y+42.5) * ratio_l
    
    total_league_shots_x, total_league_shots_y = get_total_league_shots_array(df_season_league)
    total_league_shots_x = np.abs(total_league_shots_x * ratio_w)
    total_league_shots_y = (total_league_shots_y+42.5) * ratio_l
    
    time_played_by_team = get_time_played_for_teams(df_season_league, [team_id])
    time_played_by_league = get_time_played_for_teams(df_season_league, list(range(1,56)))
    
    # put shots in bins
    xedges = list(range(0, new_width+1, 25))
    yedges = list(range(0, height+1, 25))
    H_team, xedges, yedges = np.histogram2d(total_team_shots_x, total_team_shots_y, bins=(xedges, yedges))
    H_league, xedges, yedges = np.histogram2d(total_league_shots_x, total_league_shots_y, bins=(xedges, yedges))

    # Histogram does not follow Cartesian convention (see Notes),
    # therefore transpose H for visualization purposes.
    H_team = H_team.T
    H_league = H_league.T
    
    H_team = H_team / time_played_by_team
    H_league = H_league / time_played_by_league
    shot_excess = H_team - H_league
    return shot_excess
    # min_shot_excess = shot_excess.min()
    # max_shot_excess = shot_excess.max()
    
#     fig = plt.figure(figsize=(17, 3))
#     ax = fig.add_subplot(131, title='Team shots localization')
#     plt.imshow(H_team, interpolation='nearest', origin='lower',
#         extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
    
#     fig = plt.figure(figsize=(17, 3))
#     ax = fig.add_subplot(132, title='League shots localization')
#     plt.imshow(H_league, interpolation='nearest', origin='lower',
#         extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
    
    # fig = plt.figure(figsize=(17, 3))
    # ax = fig.add_subplot(133, title='Computed team shots excess')
    # plt.imshow(H_team - H_league, interpolation='nearest', origin='lower',
    #     extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
    # plt.colorbar()
    # plt.show()

In [116]:
shot_excess = compute_team_shots_excess_by_hour_for_team(df_season_league, 4, im_new, new_width, height)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]